In [2]:
from pyspark.sql import SparkSession

Cette session Spark est configurée avec les paramètres suivants :

- `appName('examen')`: Cette méthode définit le nom de l'application Spark en cours.

- `.config("pyspark.jars", "C:/Users/anaso/anaconda3/envs/newConda/Lib/site-packages/pyspark/jars")`: On a spécifié le chemin vers le répertoire contenant les fichiers JAR nécessaires pour PySpark pour acceder a mySQL connector.

- `.getOrCreate()`: Cette méthode crée une session Spark si elle n'existe pas déjà, ou bien récupère la session existante si elle est déjà active.

In [3]:
spark = SparkSession.builder \
    .appName('examen') \
    .config("pyspark.jars", "C:/Users/anaso/anaconda3/envs/newConda/Lib/site-packages/pyspark/jars") \
    .getOrCreate()

- `mysql_props` : C'est un dictionnaire contenant les propriétés de connexion à la base de données MySQL. Les clés du dictionnaire incluent le nom d'utilisateur (`user`), le mot de passe (`password`), et le pilote JDBC à utiliser (`driver`).

- `mysql_url` : C'est l'URL de connexion à la base de données MySQL. Dans cet exemple, la base de données est hébergée localement (`localhost`) sur le port standard MySQL (`3306`). La base de données utilisée est nommée `db_exam`.

- `df_clients = spark.read.jdbc(url=mysql_url, table="clients", properties=mysql_props)` : Cette ligne de code charge les données de la table `clients` de la base de données MySQL spécifiée dans le DataFrame `df_clients`. Spark utilise le pilote JDBC pour établir la connexion, récupérer les données de la table `clients`, et les charger dans le DataFrame Spark.

- `df_commandes = spark.read.jdbc(url=mysql_url, table="commandes", properties=mysql_props)` : De manière similaire que df_clients, cette instruction charge les données de la table `commandes` de la base de données MySQL dans le DataFrame `df_commandes`.

In [4]:
mysql_props = {
    "user": "root",
    "password": "",
    "driver": "com.mysql.jdbc.Driver"
}

mysql_url = "jdbc:mysql://localhost:3306/db_exam"

df_clients = spark.read.jdbc(url=mysql_url, table="clients", properties=mysql_props)
df_commandes = spark.read.jdbc(url=mysql_url, table="commandes", properties=mysql_props)

- `df_clients.createOrReplaceTempView("clients")` : Cette methode crée une vue temporaire nommée "clients" à partir du DataFrame `df_clients`. Cette vue temporaire peut être utilisée pour exécuter des requêtes SQL sur les données de ce DataFrame.

- `df_commandes.createOrReplaceTempView("commandes")` : De manière similaire, on crée une vue temporaire nommée "commandes" à partir du DataFrame `df_commandes`. Cela permet d'accéder aux données de la table `commandes` via des requêtes SQL dans Spark SQL en utilisant cette vue temporaire.

Une fois ces vues temporaires créées, on peut effectuer des opérations SQL telles que les sélections, les filtrages, les agrégations, etc., sur les DataFrames associés en utilisant la syntaxe SQL dans Spark SQL. Ces vues temporaires servent de pont entre les DataFrames Spark et les requêtes SQL pour effectuer des analyses ou des manipulations de données.

In [5]:
df_clients.createOrReplaceTempView("clients")
df_commandes.createOrReplaceTempView("commandes")

#### 1. Affichage du nombre total de commandes passées par tous les clients

In [7]:
total_orders = spark.sql("SELECT COUNT(*) AS TotalOrders FROM commandes")
total_orders.show()

+-----------+
|TotalOrders|
+-----------+
|          5|
+-----------+



#### 2. Affichage du client qui a dépensé le plus (en terme de montant).

In [8]:
client_max_spent = spark.sql("""
    SELECT c.NOM, SUM(co.MONTANT_TOTAL) AS TotalSpent
    FROM clients c
    JOIN commandes co ON c.ID_CLIENT = co.ID_CLIENT
    GROUP BY c.NOM
    ORDER BY TotalSpent DESC
    LIMIT 1
""")
client_max_spent.show()

+------+----------+
|   NOM|TotalSpent|
+------+----------+
|Moulay|    700.49|
+------+----------+



### 3. Affichage de la moyenne des dépenses par client

In [11]:
avg_spending_per_client = spark.sql("""
    SELECT c.NOM, AVG(co.MONTANT_TOTAL) AS AvgSpending
    FROM clients c
    JOIN commandes co ON c.ID_CLIENT = co.ID_CLIENT
    GROUP BY c.NOM
""")
avg_spending_per_client.show()

+--------+-----------+
|     NOM|AvgSpending|
+--------+-----------+
|  Moulay|    350.245|
|BenAhmed|    225.625|
|  Sanlam|       75.2|
+--------+-----------+

